## Test as terminal

In [1]:
!pip freeze > requirements.txt


In [ ]:
!torchrun --standalone --nnodes=1 --nproc-per-node=8 demo_ca.py --data /test_dataset/0/webp

In [14]:
!rm models/caformer_m36-1-10000.ckpt

In [ ]:
import pd
import torch
from torch.utils.data import Dataset
import json
from PIL import Image
from PIL import ImageFile
import cv2
import boto3
import pandas as pd
import os
import numpy as np
from copy import deepcopy, copy
from torchvision import transforms
from torchvision.transforms.functional import pad
from PIL import Image
from io import BytesIO

def parse_s3_path(self, s3_path):
    """
    Parse the S3 path to get the bucket name and key.
    """
    if s3_path.startswith("s3://"):
        s3_path = s3_path[5:]
    bucket, key = s3_path.split("/", 1)
    return bucket, key 

def parse_labels(self, raw_labels):
    return raw_labels.split('|')

def generate_dict(self, df):
    tags_to_index_dict = {tag: idx for idx, tag in enumerate(df['Tags'].unique())}
    return tags_to_index_dict

metadata_path = 's3://pixai-test-uw2/richard/final_Danbooru_5.parquet'
df = pd.read_parquet(metadata_path)

In [ ]:
import os
os.getcwd()
os.mkdir('s3://pixai-test-uw2/richard/mld-ckpts/')

In [19]:
!rm models/caformer_m36-mAP45.ckpt

In [15]:
!top

=top - 20:52:52 up 3 days,  2:07,  0 users,  load average: 15.71, 12.54, 15.48
Tasks: 1051 total,  19 running, 587 sleeping,   1 stopped,   1 zombie
%Cpu(s): 19.8 us,  2.3 sy,  0.0 ni, 77.8 id,  0.0 wa,  0.0 hi,  0.1 si,  0.0 st
KiB Mem : 11762988+total, 11102728+free, 60432728 used,  5593288 buff/cache
KiB Swap:        0 total,        0 free,        0 used. 11070853+avail Mem 

  PID USER      PR  NI    VIRT    RES    SHR S  %CPU %MEM     TIME+ COMMAND     
11310 ec2-user  20   0   50.4g   3.7g 117036 R 105.9  0.3   0:12.71 python      
 7044 ec2-user  20   0   90.4g   5.0g 592960 R 100.0  0.4   1:22.78 python      
 7045 ec2-user  20   0   90.7g   5.0g 593068 R 100.0  0.4   1:23.39 python      
 7047 ec2-user  20   0   90.6g   5.0g 593808 R 100.0  0.4   1:24.80 python      
 7048 ec2-user  20   0   90.7g   5.0g 593328 R 100.0  0.4   1:21.86 python      
 7049 ec2-user  20   0   90.7g   5.0g 593956 R 100.0  0.4   1:24.49 python      
10303 ec2-user  20   0   50.4g   3.7g 117160 R 100

KeyboardInterrupt: 

In [16]:
!nvidia-smi

Wed Apr 17 20:53:06 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.12             Driver Version: 535.104.12   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          On  | 00000000:10:1C.0 Off |                    0 |
| N/A   29C    P0              74W / 400W |      4MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
!torchrun --standalone --nnodes=1 --nproc-per-node=8 train_ac_good.py --batch_size 40 --save_step 5000 --xformers False --log_step 50 







In [ ]:
torch

In [ ]:
import os
print("RANK:", os.environ.get("RANK"))
print("WORLD_SIZE:", os.environ.get("WORLD_SIZE"))
print("LOCAL_RANK:", os.environ.get("LOCAL_RANK"))


In [ ]:
!python train_ac.py --batch_size 100 --xformers False --log_step 500 --default_ckpt models/default_tresnet_l-13-22.pth --ema_ckpt models/ema_tresnet_l-13-22.pth

## Object

In [ ]:
import torch

NUM_CLASS = 100

In [ ]:
target = torch.zeros(NUM_CLASS, dtype=torch.float)
target

In [ ]:
target[[2, 3, 4, 5]]=1.
target

prepare (fake) data:

In [ ]:
import os
import unibox as ub


IMG_ROOT = "./test_dataset"

# 格式是{number}.webp
imgs = ub.traverses(IMG_ROOT, ub.IMG_FILES, relative_unix=True)
for i in range(len(imgs)):
    _img = imgs[i]
    os.system(f"mv {IMG_ROOT}/{_img} {IMG_ROOT}/{i}.webp")

In [ ]:
# labels: 格式是 list[  [number, extension, class]  ]  
import random

labels = []
for i in range(len(imgs)):
    # random pick 1~5 classes
    num_classes = random.randint(1, 5)
    _classes = random.sample(range(NUM_CLASS), num_classes)
    labels.append([i, "webp", [_classes]])

labels[:5]

In [ ]:
# save labels as a json
import json

with open("fake_labels.json", "w") as f:
    json.dump(labels, f)

In [ ]:
!python train_ac.py --batch_size 10 --xformers False --log_step 5

In [ ]:
!nvidia-smi

In [ ]:
os.getcwd()

In [ ]:
!rm models/ema_tresnet_l-2-22.pth